In [1]:
import ijson
import json


In [2]:
def ptoutlist_2invol(filename: str):
    lenofdata=15
    filedata=open(filename, 'r')
    results=[]
    datas = ijson.items(filedata, 'item')
    for data in datas:
        results.append(data)
    filedata.close()
    for key in data.keys():
        print(lenofdata)
        if "," in key:
            for y in key.split(","):
                if len(y)>lenofdata:
                    lenofdata=len(y)+1
        else:
            if len(key)>lenofdata:
                lenofdata=len(y)+1
    for key in data.keys():
        # H8 NID和REF都有
        if "," in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=(" "+key).split(",")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        else:
            print("这些键值没有输出：",key)

In [3]:


# 对输出结果记录文件进行整理
# 具有相当的特殊性，不适用于普遍情况
def ptoutlist_new(h7orh8: str,required3: bool, NIDorREF: str, srli=None, refnum=None):
    lenofdata=15
    
    datapath="../result/"
    if NIDorREF=="NID" or NIDorREF=="nid":
        if srli==None:
            print("Both sr ideal and linear ideal ? The fourth arg needed")
            return
        if h7orh8=="h7":
            if srli==None:
                print("Both sr ideal and linear ideal ? The fourth arg needed")
                return
            datapath+="NIDresultsout_h2-7_required3"+str(required3)+"_srli"+str(srli)+".txt"
        elif h7orh8=="h8":
            datapath+="NIDresultsout_h8-12_required3"+str(required3)+"_srli"+str(srli)+".txt"
        else:
            print("arg 1 wrong, it should be either \"h7\" or \"h8\"")
            return
    elif NIDorREF=="REF" or NIDorREF=="ref":
        if h7orh8=="h7":
            if refnum==None:
                print("for h11<8, refnum is needed, it should be 1,2 or 3")
                return
            datapath+="REFresultsout_h2-7_required3"+str(required3)+"_ref"+str(refnum)+".txt"
        elif h7orh8=="h8":
            if refnum!=3:
                print("for h11>7, we only compute one divisor and part of the 3 divisors reflection, so here we only count those result")
            datapath+="REFresultsout_h8-12_required3"+str(required3)+"_ref"+str(refnum)+".txt"
        else:
            print("arg 1 wrong, it should be either \"h7\" or \"h8\"")
            return
    else:
        print("arg 3 wrong, it should be either \"NID\" or \"REF\"")
        return
    
    filedata=open(datapath, 'r')
    results=json.load(filedata)

    lenall = len(results)


    for key in results[0].keys():
        listouts=[]
        # H8 NID和REF都有
        if "," in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=(" "+key).split(",")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        # 都有
        elif "N_T_F" in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=("N_T_F").split("_")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        # 都有
        elif key=="invol_divisor_type":
            # 输出结果为只需包含下列除子类型的involution
            # 0： 未讨论的其他类型除子
            # 1:  del Pezzo 1-8
            # 2:  willson
            # 3:  K3  
            # 4:  SD1
            # 5:  SD2
            # 6:  non-shrinkable rigid divisors
            # 7:  exact willson
            for _ in [1, 2, 3, 4, 5, 6, 7]:
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in [1, 2, 3, 4, 5, 6, 7]:
                    key_satisfy=[]
                    y_need=[i]
                    for keyy in invol_divisor_type.keys():
                        # 有一个对于种类的divisor就行
                        for x in keyy.split(","):
                            for y in y_need:
                                if str(y) in x:
                                    break
                            if str(y) in x:
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i-1].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str([1, 2, 3, 4, 5, 6, 7][x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")
                
            listouts=[]
            # 同样要求只包含
            # dP_n + K3 
            # dP_n + willson
            # willson + K3
            # dP_n + willson + K3
            # dP_n + exact willson
            # K3 + exact willson
            # dP_n + exact willson + K3
            for _ in [[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]]:
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in range(len([[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]])):
                    key_satisfy=[]
                    y_need=[[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]][i]
                    y_need_same=eval(str(y_need).replace("1", "6"))
                    for keyy in invol_divisor_type.keys():
                        for y in y_need:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # 注意前面是in，而这里是not in，说明所需的除子并不都在keyy里
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                            continue
                        for y in y_need_same:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # x有y，break或者，x到最后一个，break；如果y 没在x 里，说明这个里面就没有y
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str([[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]][x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")
            # 对non-shrinkable的dp divisor
            listouts=[]
            divisorneed=[]
            for x in [[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]]:
                divisorneed.append([y+(0**(y!=1))*5 for y in x])
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in range(len(divisorneed)):
                    key_satisfy=[]
                    y_need=divisorneed[i]
                    y_need_same=eval(str(y_need).replace("1", "6"))
                    for keyy in invol_divisor_type.keys():
                        for y in y_need:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # 注意前面是in，而这里是not in
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                            continue
                        for y in y_need_same:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # x有y，break或者，x到最后一个，break；如果y 没在x 里，说明这个里面就没有y
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str(divisorneed[x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")



        # NID
        elif key=="hodge_split":
            print("***************************"*2, key)
            listouts=[]
            for _ in range(12):
                # 12个空集对应，hodge-的各个情况
                listouts.append([])
            for result in results:
                # print(result["h11, polynumber, trinumber"][0])
                for listout in listouts:
                    # 每来一个result（不同的h11），就默认append一个0
                    listout.append(0)
                for keyhere in result[key]:
                    # 如果这个result有某个对应的hodge-存在，就将0覆盖
                    listouts[eval(keyhere)[1]][-1]=result["hodge_split"][keyhere]
            for i in range(12):
                # 对每一个hodge-
                outstr ="hodge_-"+ str(i)+" & "
                
                for listou in listouts[i]:
                    stroutpatch=""
                    while len(" &   "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    outstr += (str(listou)+stroutpatch+ "   & ")
                outstr += str(sum(listouts[i]))
                print(outstr)
        else:
            print(key, "这个键没有列出表格")


In [19]:
ptoutlist_new("h8", True, "ref", refnum=3)

****************************************************** h11, polynumber, triangnumber
 h11           &  8           &  9           &  10          &  11          &  12           & 50    replaceitto
 polynumber    &  1486        &  1581        &  1554        &  1621        &  1692         & 7934    replaceitto
 triangnumber  &  6847        &  9833        &  12185       &  15638       &  14980        & 59483    replaceitto
****************************************************** numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber
 numo3_ref     &  455         &  449         &  100         &  71          &  304          & 1379    replaceitto
 numo7_ref     &  2153        &  2478        &  2439        &  1836        &  2058         & 10964    replaceitto
 numo3o7_ref   &  21752       &  28793       &  29532       &  36998       &  43727        & 160802    replaceitto
 numo5_ref     &  0           &  0           &  0           &  0           &  0            & 0    

In [ ]:
# 输出结果为只需包含下列除子类型的involution
# 0： 未讨论的其他类型除子
# 1:  del Pezzo 1-8
# 2:  willson
# 3:  K3  
# 4:  SD1
# 5:  SD2
# 6:  non-shrinkable rigid divisors
# 7:  exact willson

In [10]:
ptoutlist_new("h8", False, "NID", srli=False)

****************************************************** h11, polynumber, trinumber
 h11           &  8           &  9           &  10          &  11          &  12           & 50    replaceitto
 polynumber    &  68          &  65          &  48          &  41          &  44           & 266    replaceitto
 trinumber     &  243         &  256         &  210         &  305         &  294          & 1308    replaceitto
****************************************************** hodge_split
hodge_-0 & 0            & 0            & 0            & 0            & 0            & 0
hodge_-1 & 159          & 169          & 122          & 110          & 54           & 614
hodge_-2 & 71           & 88           & 34           & 61           & 50           & 304
hodge_-3 & 27           & 29           & 18           & 96           & 167          & 337
hodge_-4 & 2            & 32           & 11           & 82           & 31           & 158
hodge_-5 & 0            & 0            & 29           & 2          

In [15]:
sigma=[[8,9],[6,7],[3,4],[2,5],[0,1]]

outsr="{"
for srlis in sigma:
    outstr="{"
    for x in srlis:
        outstr+="x_"+str(x+1)+", "
    outstr=outstr[:-2]+"},"
    outsr+=outstr

outsr=outsr[:-1]+"}"
print(outsr)

{{x_9, x_10},{x_7, x_8},{x_4, x_5},{x_3, x_6},{x_1, x_2}}


In [1]:
srlist=[[1,4,6],[1,4,7],[1,6,7],[1,8],[1,9],
 [2,3,8],[2,3,10],[2,5,8],[2,5,10], 
[2,7],[3,5,8],[3,5,10],[3,6],[4,5],
 [4,6,9],[4,7,9],[6,7,9],[8,10],
 [9,10]]

outsr="{"
for srlis in srlist:
    outstr="{"
    for x in srlis:
        outstr+="x_"+str(x)+", "
    outstr=outstr[:-2]+"},"
    outsr+=outstr

outsr=outsr[:-1]+"}"
print(outsr)

{{x_1, x_4, x_6},{x_1, x_4, x_7},{x_1, x_6, x_7},{x_1, x_8},{x_1, x_9},{x_2, x_3, x_8},{x_2, x_3, x_10},{x_2, x_5, x_8},{x_2, x_5, x_10},{x_2, x_7},{x_3, x_5, x_8},{x_3, x_5, x_10},{x_3, x_6},{x_4, x_5},{x_4, x_6, x_9},{x_4, x_7, x_9},{x_6, x_7, x_9},{x_8, x_10},{x_9, x_10}}


In [3]:
str1="""22x0^2x1^2x2^2x4^2x8^2 + 40x0x1^2x2^2x3x4x7x8^2 - 62x0x1^2x2x4^2x5x7x8^2 + 77x0x1x2^2x4^2x6x7x8^2 + 69x1^2x2^2x3^2x7^2x8^2 + 61x1^2x2x3x4x5x7^2x8^2 + 65x1^2x4^2x5^2x7^2x8^2 - 51x1x2^2x3x4x6x7^2x8^2 - 17x1x2x4^2x5x6x7^2x8^2 + 19x2^2x4^2x6^2x7^2x8^2 + 47x0^2x1^2x2^2x3^2x8x9 - 12x0^2x1^2x2x3x4x5x8x9 + 47x0^2x1^2x4^2x5^2x8x9 + 14x0^2x1x2^2x3x4x6x8x9 - 5x0^2x1x2x4^2x5x6x8x9 - 51x0^2x2^2x4^2x6^2x8x9 - 5x0x1^2x2x3^2x5x7x8x9 + 14x0x1^2x3x4x5^2x7x8x9 - 42x0x1x2^2x3^2x6x7x8x9 + 70x0x1x2x3x4x5x6x7x8x9 - 42x0x1x4^2x5^2x6x7x8x9 - 72x0x2^2x3x4x6^2x7x8x9 + 16x0x2x4^2x5x6^2x7x8x9 - 51x1^2x3^2x5^2x7^2x8x9 + 16x1x2x3^2x5x6x7^2x8x9 - 72x1x3x4x5^2x6x7^2x8x9 - 74x2^2x3^2x6^2x7^2x8x9 - 11x2x3x4x5x6^2x7^2x8x9 - 74x4^2x5^2x6^2x7^2x8x9 + 22x0^2x1^2x3^2x5^2x9^2 - 62x0^2x1x2x3^2x5x6x9^2 + 40x0^2x1x3x4x5^2x6x9^2 + 65x0^2x2^2x3^2x6^2x9^2 + 61x0^2x2x3x4x5x6^2x9^2 + 69x0^2x4^2x5^2x6^2x9^2 + 77x0x1x3^2x5^2x6x7x9^2 - 17x0x2x3^2x5x6^2x7x9^2 - 51x0x3x4x5^2x6^2x7x9^2 + 19x3^2x5^2x6^2x7^2x9^2"""

In [23]:
outstr=""
numadd1=False
for x in "[[x0 - 1, x1 - 1, x2 - 1, x3 - 1, x5 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x3 - 1, x6 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x3 - 1, x7 - 1, x8 - 1], [x0 - 1, x1 - 1, x2 - 1, x3 - 1, x8 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x7 - 1, x8 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x8 - 1, x9 - 1], [x0 - 1, x1 - 1, x3 - 1, x4 - 1, x5 - 1, x9 - 1], [x0 - 1, x1 - 1, x3 - 1, x5 - 1, x7 - 1, x9 - 1], [x0 - 1, x1 - 1, x4 - 1, x5 - 1, x6 - 1, x9 - 1], [x0 - 1, x1 - 1, x4 - 1, x5 - 1, x7 - 1, x8 - 1], [x0 - 1, x1 - 1, x4 - 1, x5 - 1, x8 - 1, x9 - 1], [x0 - 1, x2 - 1, x3 - 1, x4 - 1, x6 - 1, x9 - 1], [x0 - 1, x2 - 1, x3 - 1, x6 - 1, x7 - 1, x9 - 1], [x0 - 1, x2 - 1, x4 - 1, x5 - 1, x6 - 1, x9 - 1], [x0 - 1, x2 - 1, x4 - 1, x6 - 1, x7 - 1, x8 - 1], [x0 - 1, x2 - 1, x4 - 1, x6 - 1, x8 - 1, x9 - 1], [x0 - 1, x3 - 1, x5 - 1, x6 - 1, x7 - 1, x9 - 1], [x0 - 1, x4 - 1, x5 - 1, x6 - 1, x7 - 1, x9 - 1], [x1 - 1, x2 - 1, x3 - 1, x5 - 1, x7 - 1, x8 - 1], [x1 - 1, x2 - 1, x3 - 1, x6 - 1, x7 - 1, x8 - 1], [x1 - 1, x3 - 1, x4 - 1, x5 - 1, x7 - 1, x8 - 1], [x1 - 1, x3 - 1, x5 - 1, x7 - 1, x8 - 1, x9 - 1], [x1 - 1, x4 - 1, x5 - 1, x6 - 1, x7 - 1, x8 - 1], [x2 - 1, x3 - 1, x4 - 1, x6 - 1, x7 - 1, x8 - 1], [x2 - 1, x3 - 1, x6 - 1, x7 - 1, x8 - 1, x9 - 1], [x2 - 1, x4 - 1, x5 - 1, x6 - 1, x7 - 1, x8 - 1], [x3 - 1, x5 - 1, x6 - 1, x7 - 1, x8 - 1, x9 - 1], [x4 - 1, x5 - 1, x6 - 1, x7 - 1, x8 - 1, x9 - 1]]":
    if x=="x":
        if numadd1:
            outstr+=str(int(num)+1)+" "+x+"_"
        else:
            outstr+=" "+x+"_"
            numadd1=True
        num=""
    elif x=="^" or x==" " or x=="," or x=="*":
        if numadd1:
            numadd1=False
            outstr+=str(int(num)+1)+""+x
        else:
            outstr+=x
    else:
        if numadd1:
            num+=x
        else:
            outstr+=x
if numadd1:
    outstr+=str(int(num)+1)
print(outstr.replace(" ", "").replace("[","<").replace("]",">"))

<<x_1-1,x_2-1,x_3-1,x_4-1,x_6-1,x_10-1>,<x_1-1,x_2-1,x_3-1,x_4-1,x_7-1,x_10-1>,<x_1-1,x_2-1,x_3-1,x_4-1,x_8-1,x_9-1>,<x_1-1,x_2-1,x_3-1,x_4-1,x_9-1,x_10-1>,<x_1-1,x_2-1,x_3-1,x_5-1,x_8-1,x_9-1>,<x_1-1,x_2-1,x_3-1,x_5-1,x_9-1,x_10-1>,<x_1-1,x_2-1,x_4-1,x_5-1,x_6-1,x_10-1>,<x_1-1,x_2-1,x_4-1,x_6-1,x_8-1,x_10-1>,<x_1-1,x_2-1,x_5-1,x_6-1,x_7-1,x_10-1>,<x_1-1,x_2-1,x_5-1,x_6-1,x_8-1,x_9-1>,<x_1-1,x_2-1,x_5-1,x_6-1,x_9-1,x_10-1>,<x_1-1,x_3-1,x_4-1,x_5-1,x_7-1,x_10-1>,<x_1-1,x_3-1,x_4-1,x_7-1,x_8-1,x_10-1>,<x_1-1,x_3-1,x_5-1,x_6-1,x_7-1,x_10-1>,<x_1-1,x_3-1,x_5-1,x_7-1,x_8-1,x_9-1>,<x_1-1,x_3-1,x_5-1,x_7-1,x_9-1,x_10-1>,<x_1-1,x_4-1,x_6-1,x_7-1,x_8-1,x_10-1>,<x_1-1,x_5-1,x_6-1,x_7-1,x_8-1,x_10-1>,<x_2-1,x_3-1,x_4-1,x_6-1,x_8-1,x_9-1>,<x_2-1,x_3-1,x_4-1,x_7-1,x_8-1,x_9-1>,<x_2-1,x_4-1,x_5-1,x_6-1,x_8-1,x_9-1>,<x_2-1,x_4-1,x_6-1,x_8-1,x_9-1,x_10-1>,<x_2-1,x_5-1,x_6-1,x_7-1,x_8-1,x_9-1>,<x_3-1,x_4-1,x_5-1,x_7-1,x_8-1,x_9-1>,<x_3-1,x_4-1,x_7-1,x_8-1,x_9-1,x_10-1>,<x_3-1,x_5-1,x_6-1,x_7-1,x_8-1,x_

In [9]:
outstr=""
numadd1=False
for x in str1:
    if x=="x":
        if numadd1:
            outstr+=str(int(num)+1)+"} "+x+"_{"
        else:
            outstr+=" "+x+"_{"
            numadd1=True
        num=""
    elif x=="^" or x==" ":
        if numadd1:
            numadd1=False
            outstr+=str(int(num)+1)+"}"+x
        else:
            outstr+=x
    else:
        if numadd1:
            num+=x
        else:
            outstr+=x
print(outstr)

22 x_{1}^2 x_{2}^2 x_{3}^2 x_{5}^2 x_{9}^2 + 40 x_{1} x_{2}^2 x_{3}^2 x_{4} x_{5} x_{8} x_{9}^2 - 62 x_{1} x_{2}^2 x_{3} x_{5}^2 x_{6} x_{8} x_{9}^2 + 77 x_{1} x_{2} x_{3}^2 x_{5}^2 x_{7} x_{8} x_{9}^2 + 69 x_{2}^2 x_{3}^2 x_{4}^2 x_{8}^2 x_{9}^2 + 61 x_{2}^2 x_{3} x_{4} x_{5} x_{6} x_{8}^2 x_{9}^2 + 65 x_{2}^2 x_{5}^2 x_{6}^2 x_{8}^2 x_{9}^2 - 51 x_{2} x_{3}^2 x_{4} x_{5} x_{7} x_{8}^2 x_{9}^2 - 17 x_{2} x_{3} x_{5}^2 x_{6} x_{7} x_{8}^2 x_{9}^2 + 19 x_{3}^2 x_{5}^2 x_{7}^2 x_{8}^2 x_{9}^2 + 47 x_{1}^2 x_{2}^2 x_{3}^2 x_{4}^2 x_{9} x_{10} - 12 x_{1}^2 x_{2}^2 x_{3} x_{4} x_{5} x_{6} x_{9} x_{10} + 47 x_{1}^2 x_{2}^2 x_{5}^2 x_{6}^2 x_{9} x_{10} + 14 x_{1}^2 x_{2} x_{3}^2 x_{4} x_{5} x_{7} x_{9} x_{10} - 5 x_{1}^2 x_{2} x_{3} x_{5}^2 x_{6} x_{7} x_{9} x_{10} - 51 x_{1}^2 x_{3}^2 x_{5}^2 x_{7}^2 x_{9} x_{10} - 5 x_{1} x_{2}^2 x_{3} x_{4}^2 x_{6} x_{8} x_{9} x_{10} + 14 x_{1} x_{2}^2 x_{4} x_{5} x_{6}^2 x_{8} x_{9} x_{10} - 42 x_{1} x_{2} x_{3}^2 x_{4}^2 x_{7} x_{8} x_{9} x_{10} + 70 x_{

In [ ]:
data={
  "lookout": [
    false,
    [
      9.0,
      9.0,
      9.0,
      18.0,
      54.0,
      9.0,
      9.0,
      0.0,
      9.0,
      0.0,
      0.0
    ]
  ],
  "BP_not_in_F": "[[-2, -3, -2, -2], [0, 1, 0, 0], [0, 1, 0, 2], [0, 1, 2, 0], [1, 0, 0, 0], [-1, -1, -1, -1], [-1, -1, -1, 0], [-1, -1, 0, -1], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1]]",
  "F_Intsec": "[[[[2, 0, 0, 0, 0, 0, 0], [0, -2, 0, 0, 0, 0, 0], [0, 0, -2, 0, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, -2, 0, 0, 0, 0, 0], [-2, -3, -1, 0, -1, 0, 0], [0, -1, -1, 0, 1, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, -1, 1, 0, -1, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, -2, 0, 0, 0, 0], [0, -1, -1, 0, 1, 0, 0], [-2, -1, -3, 0, -1, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 1, -1, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, -2, 0, 0], [0, -1, 1, 0, -1, 0, 0], [0, 1, -1, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-2, -1, -1, 0, -3, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]]], [[[0, -2, 0, 0, 0, 0, 0], [-2, -3, -1, 0, -1, 0, 0], [0, -1, -1, 0, 1, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, -1, 1, 0, -1, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, -3, -1, 0, -1, 0, 0], [-3, -3.5, -1.5, 0, -1.5, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, -1, -1, 0, 1, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [0, 0, 0, -2, 0, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, -1, 1, 0, -1, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]], [[[0, 0, -2, 0, 0, 0, 0], [0, -1, -1, 0, 1, 0, 0], [-2, -1, -3, 0, -1, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 1, -1, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, -1, -1, 0, 1, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [0, 0, 0, -2, 0, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, -1, -3, 0, -1, 0, 0], [-1, -1.5, -1.5, 0, 0.5, 0, 0], [-3, -1.5, -3.5, 0, -1.5, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 1, -1, 0, -1, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]]], [[[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, -2, -2, 0, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [-2, -2, -2, 0, 0, 0, 0], [0, 0, 0, -2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]], [[[0, 0, 0, 0, -2, 0, 0], [0, -1, 1, 0, -1, 0, 0], [0, 1, -1, 0, -1, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-2, -1, -1, 0, -3, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, -1, 1, 0, -1, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 1, -1, 0, -1, 0, 0], [1, 0.5, 0.5, 0, 0.5, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, -1, -1, 0, -3, 0, 0], [-1, -1.5, 0.5, 0, -1.5, 0, 0], [-1, 0.5, -1.5, 0, -1.5, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-3, -1.5, -1.5, 0, -3.5, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, -2]], [[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]]], [[[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, -2, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, -2, 0, 0, -2, 0, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-2, -2, 0, 0, -2, 0, 0], [0, 0, 0, 0, 0, -2, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]], [[[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[-2, 0, -2, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0], [-2, 0, -2, 0, -2, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -2]]]]",
  "invol1": [
    "[0] ; finedata ; O3: 3 [x5, x6, x8], O7: 9 [x0], O7: 9 [x1], O7: 9 [x2], O7: 18 [x3], satisfied 48; [h12p, h12n, smooth]: [11.0, 26.0, True]",
    "[1] ; finedata ; O3: 3 [x5, x6, x8], O7: 9 [x1], O7: 9 [x0], O7: 9 [x2], O7: 18 [x3], satisfied 48; [h12p, h12n, smooth]: [11.0, 26.0, True]",
    "[2] ; finedata ; O3: 3 [x5, x6, x8], O7: 9 [x2], O7: 9 [x0], O7: 9 [x1], O7: 18 [x3], satisfied 48; [h12p, h12n, smooth]: [11.0, 26.0, True]",
    "[3] ; finedata ; O3: 3 [x5, x6, x8], O7: 18 [x3], O7: 9 [x0], O7: 9 [x1], O7: 9 [x2], satisfied 48; [h12p, h12n, smooth]: [11.0, 26.0, True]",
    "[4] ; finedata ; O3: 1.0 [x0, x5, x6, x7], O3: 1.0 [x1, x5, x8, x9], O3: 1.0 [x2, x6, x8, x10], O3: 1.0 [x3, x5, x6, x7], O3: 1.0 [x3, x5, x8, x9], O3: 1.0 [x3, x6, x8, x10], O7: 54 [x4], satisfied 60.0; [h12p, h12n, smooth]: [8.0, 29.0, True]",
    "[5] ; finedata ; O3: 3 [x2, x6, x8], O3: 1.0 [x0, x4, x6, x7], O3: 1.0 [x1, x4, x8, x9], O3: 1.0 [x3, x4, x6, x7], O3: 1.0 [x3, x4, x8, x9], O7: 9 [x5], O7: 4.0 [x4, x10], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[6] ; finedata ; O3: 3 [x1, x5, x8], O3: 1.0 [x0, x4, x5, x7], O3: 1.0 [x2, x4, x8, x10], O3: 1.0 [x3, x4, x5, x7], O3: 1.0 [x3, x4, x8, x10], O7: 9 [x6], O7: 4.0 [x4, x9], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[7] ; finedata ; O3: 3 [x1, x5, x9], O3: 3 [x2, x6, x10], O3: 3 [x3, x5, x9], O3: 3 [x3, x6, x10], O3: 1.0 [x0, x4, x5, x6], O7: 0 [x7], O7: 7.0 [x4, x8], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[8] ; finedata ; O3: 3 [x0, x5, x6], O3: 1.0 [x1, x4, x5, x9], O3: 1.0 [x2, x4, x6, x10], O3: 1.0 [x3, x4, x5, x9], O3: 1.0 [x3, x4, x6, x10], O7: 9 [x8], O7: 4.0 [x4, x7], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[9] ; finedata ; O3: 3 [x0, x5, x7], O3: 3 [x2, x8, x10], O3: 3 [x3, x5, x7], O3: 3 [x3, x8, x10], O3: 1.0 [x1, x4, x5, x8], O7: 0 [x9], O7: 7.0 [x4, x6], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[10] ; finedata ; O3: 3 [x0, x6, x7], O3: 3 [x1, x8, x9], O3: 3 [x3, x6, x7], O3: 3 [x3, x8, x9], O3: 1.0 [x2, x4, x6, x8], O7: 0 [x10], O7: 7.0 [x4, x5], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]"
  ],
  "KK_data": "4 5  M:12 5 N:40 5 H:37,7 [60]",
  "Linear_I": [
    "1/2*z5 - 1/2*z6 - 1/2*z7 - 1/2*z8",
    "1/2*z5 - 1/2*z6 - 1/2*z9 - 1/2*z10",
    "1/2*z5 - 1/2*z7 - 1/2*z9 - 1/2*z11",
    "1/2*z5 - 1/2*z8 - 1/2*z10 - 1/2*z11",
    "z5",
    "z6",
    "z7",
    "z8",
    "z9",
    "z10",
    "z11"
  ],
  "NVERTS": "[[-1, -1, 0, 0], [-1, -1, 0, 3], [-1, -1, 3, 0], [-1, 5, -3, -3], [2, -1, 0, 0]]",
  "SR_list": "[[1, 2], [1, 3], [1, 4], [1, 9], [1, 10], [1, 11], [2, 3], [2, 4], [2, 7], [2, 8], [2, 11], [3, 4], [3, 6], [3, 8], [3, 10], [4, 5, 6, 7], [4, 5, 6, 9], [4, 5, 7, 9], [5, 6, 7, 8], [5, 6, 9, 10], [5, 7, 9, 11], [6, 11], [7, 10], [8, 9], [8, 10], [8, 11], [10, 11]]",
  "Sectors": "[[x0 - 1, x1 - 1, x2 - 1, x3 - 1, x7 - 1, x9 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x5 - 1, x7 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x6 - 1, x7 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x7 - 1, x9 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x4 - 1, x8 - 1, x9 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x5 - 1, x6 - 1, x7 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x5 - 1, x6 - 1, x7 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x5 - 1, x7 - 1, x8 - 1, x9 - 1], [x0 - 1, x1 - 1, x2 - 1, x5 - 1, x8 - 1, x9 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x6 - 1, x7 - 1, x8 - 1, x10 - 1], [x0 - 1, x1 - 1, x2 - 1, x6 - 1, x8 - 1, x9 - 1, x10 - 1], [x0 - 1, x1 - 1, x3 - 1, x4 - 1, x5 - 1, x7 - 1, x9 - 1], [x0 - 1, x1 - 1, x3 - 1, x5 - 1, x6 - 1, x7 - 1, x9 - 1], [x0 - 1, x1 - 1, x3 - 1, x5 - 1, x7 - 1, x8 - 1, x9 - 1], [x0 - 1, x1 - 1, x3 - 1, x5 - 1, x7 - 1, x9 - 1, x10 - 1], [x0 - 1, x2 - 1, x3 - 1, x4 - 1, x6 - 1, x7 - 1, x10 - 1], [x0 - 1, x2 - 1, x3 - 1, x5 - 1, x6 - 1, x7 - 1, x10 - 1], [x0 - 1, x2 - 1, x3 - 1, x6 - 1, x7 - 1, x8 - 1, x10 - 1], [x0 - 1, x2 - 1, x3 - 1, x6 - 1, x7 - 1, x9 - 1, x10 - 1], [x1 - 1, x2 - 1, x3 - 1, x4 - 1, x8 - 1, x9 - 1, x10 - 1], [x1 - 1, x2 - 1, x3 - 1, x5 - 1, x8 - 1, x9 - 1, x10 - 1], [x1 - 1, x2 - 1, x3 - 1, x6 - 1, x8 - 1, x9 - 1, x10 - 1], [x1 - 1, x2 - 1, x3 - 1, x7 - 1, x8 - 1, x9 - 1, x10 - 1]]",
  "Wmatrix2": "[[-6  1 -1  0  0  2  0  0  0  2  2  0]\n [ 0  0  1  1  0  0  0  0  0 -2  0  0]\n [ 0  0  1  0  1  0  0  0  0  0 -2  0]\n [-3  0 -1  0  0  1  1  0  0  1  1  0]\n [-3  0  0  0  0  1  0  1  0  0  1  0]\n [-3  0  0  0  0  1  0  0  1  1  0  0]\n [ 0  0  1  0  0  0  0  0  0 -1 -1  1]]",
  "divisor_indenp": "['z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11']",
  "divisor": "[[1 list([1, 0, 0, 7]) 9 216.0 1.0]\n [2 list([1, 0, 0, 7]) 9 216.0 1.0]\n [3 list([1, 0, 0, 7]) 9 216.0 1.0]\n [4 list([1, 0, 0, 16]) 18 -288.0 1.0]\n [5 list([1, 0, 4, 44]) 54 576.0 5.0]\n [6 list([1, 1, 0, 11]) 9 -504.0 0.0]\n [7 list([1, 1, 0, 11]) 9 -504.0 0.0]\n [8 list([1, 1, 0, 2]) 0 0.0 0.0]\n [9 list([1, 1, 0, 11]) 9 -504.0 0.0]\n [10 list([1, 1, 0, 2]) 0 0.0 0.0]\n [11 list([1, 1, 0, 2]) 0 0.0 0.0]]",
  "label": 6,
  "polyid": 545,
  "tri_id": 5,
  "triple_inters": "{(1, 5, 7): 3, (1, 6, 7): 3, (1, 5, 6): 3, (1, 5, 8): 3, (1, 6, 8): 3, (1, 7, 8): 3, (2, 5, 9): 3, (2, 6, 9): 3, (2, 5, 6): 3, (2, 6, 10): 3, (2, 5, 10): 3, (2, 9, 10): 3, (3, 7, 9): 3, (3, 5, 7): 3, (3, 5, 9): 3, (3, 7, 11): 3, (3, 5, 11): 3, (3, 9, 11): 3, (4, 6, 8): 3, (4, 5, 8): 3, (4, 6, 10): 3, (4, 5, 10): 3, (4, 7, 8): 3, (4, 7, 11): 3, (4, 5, 11): 3, (4, 9, 10): 3, (4, 9, 11): 3, (6, 7, 9): 3, (1, 1, 1): 3, (1, 1, 5): -3, (1, 1, 6): -3, (1, 1, 7): -3, (1, 1, 8): -3, (1, 5, 5): 3, (1, 6, 6): 3, (1, 7, 7): 3, (1, 8, 8): 3, (5, 5, 5): 6, (5, 6, 6): -6, (5, 7, 7): -6, (5, 8, 8): -6, (6, 6, 6): -9, (6, 6, 7): -3, (6, 7, 7): -3, (6, 8, 8): -6, (7, 7, 7): -9, (7, 8, 8): -6, (2, 2, 2): 3, (2, 2, 5): -3, (2, 2, 6): -3, (2, 2, 9): -3, (2, 2, 10): -3, (2, 5, 5): 3, (2, 6, 6): 3, (2, 9, 9): 3, (2, 10, 10): 3, (5, 9, 9): -6, (5, 10, 10): -6, (6, 6, 9): -3, (6, 9, 9): -3, (6, 10, 10): -6, (9, 9, 9): -9, (9, 10, 10): -6, (3, 3, 3): 3, (3, 3, 5): -3, (3, 3, 7): -3, (3, 3, 9): -3, (3, 3, 11): -3, (3, 5, 5): 3, (3, 7, 7): 3, (3, 9, 9): 3, (3, 11, 11): 3, (5, 11, 11): -6, (7, 7, 9): -3, (7, 9, 9): -3, (7, 11, 11): -6, (9, 11, 11): -6, (4, 4, 4): -6, (4, 4, 5): -3, (4, 4, 6): -3, (4, 4, 7): -3, (4, 4, 8): 3, (4, 4, 9): -3, (4, 4, 10): 3, (4, 4, 11): 3, (4, 5, 5): 3, (4, 6, 6): -3, (4, 7, 7): -3, (4, 8, 8): -3, (4, 9, 9): -3, (4, 10, 10): -3, (4, 11, 11): -3, (0, 1, 7): 9, (0, 1, 5): 9, (0, 1, 6): 9, (0, 1, 8): 9, (0, 2, 5): 9, (0, 2, 9): 9, (0, 2, 6): 9, (0, 2, 10): 9, (0, 3, 7): 9, (0, 3, 9): 9, (0, 3, 5): 9, (0, 3, 11): 9, (0, 4, 8): 9, (0, 4, 5): 9, (0, 4, 10): 9, (0, 4, 11): 9, (0, 1, 1): -9, (0, 5, 5): 18, (0, 6, 6): -18, (0, 7, 7): -18, (0, 8, 8): -18, (0, 2, 2): -9, (0, 9, 9): -18, (0, 10, 10): -18, (0, 3, 3): -9, (0, 11, 11): -18, (0, 4, 4): -9, (0, 0, 1): 27, (0, 0, 5): 54, (0, 0, 2): 27, (0, 0, 3): 27, (0, 0, 4): 27, (0, 0, 0): 162}",
  "invol2": [
    "[6, 8] ; finedata ; O5: [x6, x8], O5: [x0, x5], O5: [x1, x5], O5: [x3, x5], O5: [x2, x4, x10], O5: [x3, x4, x10], O5: [x4, x6, x7], O5: [x4, x8, x9], No_o3_o7 ",
    "[0, 1] ; finedata ; [[x0], [x1], [x2], [x3], [x4], [x5], [x6], [x7], [x8], [x9], [x10]] total fixed;",
    "[1, 5] ; finedata ; O5: [x1, x5], O5: [x0, x5], O5: [x3, x5], O5: [x6, x8], O5: [x2, x4, x10], O5: [x3, x4, x10], O5: [x4, x6, x7], O5: [x4, x8, x9], No_o3_o7 ",
    "[2, 9] ; finedata ; O5: [x1, x9], O5: [x3, x9], O5: [x5, x7], O5: [x8, x10], O5: [x0, x4, x6], O5: [x2, x4, x6], O5: [x3, x4, x6], O5: [x4, x5, x8], No_o3_o7 ",
    "[2, 5] ; finedata ; O5: [x0, x5], O5: [x1, x5], O5: [x3, x5], O5: [x6, x8], O5: [x2, x4, x10], O5: [x3, x4, x10], O5: [x4, x6, x7], O5: [x4, x8, x9], No_o3_o7 ",
    "[0, 2] ; finedata ; [[x0], [x1], [x2], [x3], [x4], [x5], [x6], [x7], [x8], [x9], [x10]] total fixed;",
    "[1, 4] ; finedata ; O5: [x1, x4], O5: [x0, x4], O5: [x2, x4], O5: [x3, x4], O5: [x5, x6, x7], O5: [x5, x8, x9], O5: [x6, x8, x10], No_o3_o7 ",
    "[0, 6] ; finedata ; O5: [x0, x6], O5: [x2, x6], O5: [x3, x6], O5: [x5, x8], O5: [x1, x4, x9], O5: [x3, x4, x9], O5: [x4, x5, x7], O5: [x4, x8, x10], No_o3_o7 ",
    "[5, 6] ; finedata ; O5: [x5, x6], O5: [x1, x8], O5: [x2, x8], O5: [x3, x8], O5: [x4, x9], O5: [x0, x4, x7], O5: [x3, x4, x7], O5: [x4, x6, x10], No_o3_o7 ",
    "[4, 9] ; finedata ; O5: [x4, x9], O5: [x1, x5, x8], O5: [x3, x5, x8], O9: [x6], No_o3_o7 ",
    "[0, 4] ; finedata ; O5: [x0, x4], O5: [x1, x4], O5: [x2, x4], O5: [x3, x4], O5: [x5, x6, x7], O5: [x5, x8, x9], O5: [x6, x8, x10], No_o3_o7 ",
    "[4, 5] ; finedata ; O5: [x4, x5], O5: [x0, x6, x7], O5: [x1, x8, x9], O5: [x3, x6, x7], O5: [x3, x8, x9], O9: [x10], No_o3_o7 ",
    "[2, 4] ; finedata ; O5: [x2, x4], O5: [x0, x4], O5: [x1, x4], O5: [x3, x4], O5: [x5, x6, x7], O5: [x5, x8, x9], O5: [x6, x8, x10], No_o3_o7 ",
    "[0, 7] ; finedata ; O5: [x0, x7], O5: [x3, x7], O5: [x5, x9], O5: [x6, x10], O5: [x1, x4, x8], O5: [x2, x4, x8], O5: [x3, x4, x8], O5: [x4, x5, x6], No_o3_o7 ",
    "[1, 6] ; finedata ; O5: [x0, x6], O5: [x2, x6], O5: [x3, x6], O5: [x5, x8], O5: [x1, x4, x9], O5: [x3, x4, x9], O5: [x4, x5, x7], O5: [x4, x8, x10], No_o3_o7 "
  ],
  "invol3": [
    "[3, 4, 5] ; finedata ; O3: 3 [x0, x4, x5], O3: 3 [x1, x4, x5], O7: 3.0 [x2, x10], O7: 3.0 [x3, x10], O7: 4.0 [x6, x7], O7: 4.0 [x8, x9], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[2, 7, 8] ; finedata ; O3: 1.0 [x4, x5, x6, x8], O7: 3.0 [x0, x4], O7: 3.0 [x1, x4], O7: 3.0 [x2, x4], O7: 6.0 [x3, x4], satisfied 16.0; [h12p, h12n, smooth]: [19.0, 18.0, True]",
    "[3, 6, 10] ; finedata ; O3: 3 [x3, x6, x10], O3: 3 [x1, x5, x9], O3: 3 [x2, x6, x10], O3: 3 [x3, x5, x9], O3: 1.0 [x0, x4, x5, x6], O7: 0 [x7], O7: 7.0 [x4, x8], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[0, 5, 6] ; finedata ; O3: 3 [x0, x5, x6], O3: 1.0 [x1, x4, x5, x9], O3: 1.0 [x2, x4, x6, x10], O3: 1.0 [x3, x4, x5, x9], O3: 1.0 [x3, x4, x6, x10], O7: 9 [x8], O7: 4.0 [x4, x7], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[4, 6, 10] ; finedata ; O3: 3 [x0, x4, x7], O3: 3 [x3, x4, x7], O7: 3.0 [x1, x8], O7: 3.0 [x2, x8], O7: 4.5 [x3, x8], O7: 4.5 [x5, x6], fail_to_d3_tadpole 21.0; [h12p, h12n, smooth]: [17.75, 19.25, False]",
    "[1, 6, 10] ; finedata ; O3: 3 [x1, x5, x9], O3: 3 [x2, x6, x10], O3: 3 [x3, x5, x9], O3: 3 [x3, x6, x10], O3: 1.0 [x0, x4, x5, x6], O7: 0 [x7], O7: 7.0 [x4, x8], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[4, 5, 9] ; finedata ; O3: 3 [x0, x4, x7], O3: 3 [x3, x4, x7], O7: 3.0 [x1, x8], O7: 3.0 [x2, x8], O7: 4.5 [x3, x8], O7: 4.5 [x5, x6], fail_to_d3_tadpole 21.0; [h12p, h12n, smooth]: [17.75, 19.25, False]",
    "[1, 4, 7] ; finedata ; O3: 3 [x0, x4, x7], O3: 3 [x3, x4, x7], O7: 3.0 [x1, x8], O7: 3.0 [x2, x8], O7: 4.5 [x3, x8], O7: 4.5 [x5, x6], fail_to_d3_tadpole 21.0; [h12p, h12n, smooth]: [17.75, 19.25, False]",
    "[0, 8, 10] ; finedata ; O3: 3 [x0, x5, x7], O3: 3 [x2, x8, x10], O3: 3 [x3, x5, x7], O3: 3 [x3, x8, x10], O3: 1.0 [x1, x4, x5, x8], O7: 0 [x9], O7: 7.0 [x4, x6], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[4, 7, 8] ; finedata ; [[x0], [x1], [x2], [x3], [x4], [x5], [x6], [x7], [x8], [x9], [x10]] total fixed;",
    "[0, 4, 8] ; finedata ; O3: 3 [x1, x4, x8], O3: 3 [x2, x4, x8], O7: 3.0 [x0, x7], O7: 3.0 [x3, x7], O7: 4.0 [x5, x9], O7: 4.0 [x6, x10], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[2, 5, 7] ; finedata ; O3: 3 [x0, x5, x7], O3: 3 [x2, x8, x10], O3: 3 [x3, x5, x7], O3: 3 [x3, x8, x10], O3: 1.0 [x1, x4, x5, x8], O7: 0 [x9], O7: 7.0 [x4, x6], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[0, 4, 6] ; finedata ; O3: 3 [x0, x4, x6], O3: 3 [x2, x4, x6], O7: 3.0 [x1, x9], O7: 3.0 [x3, x9], O7: 4.0 [x5, x7], O7: 4.0 [x8, x10], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[1, 3, 5] ; finedata ; O3: 3 [x2, x6, x8], O3: 1.0 [x0, x4, x6, x7], O3: 1.0 [x1, x4, x8, x9], O3: 1.0 [x3, x4, x6, x7], O3: 1.0 [x3, x4, x8, x9], O7: 9 [x5], O7: 4.0 [x4, x10], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]",
    "[3, 5, 6] ; finedata ; O3: 3 [x0, x5, x6], O3: 1.0 [x1, x4, x5, x9], O3: 1.0 [x2, x4, x6, x10], O3: 1.0 [x3, x4, x5, x9], O3: 1.0 [x3, x4, x6, x10], O7: 9 [x8], O7: 4.0 [x4, x7], satisfied 20.0; [h12p, h12n, smooth]: [18.0, 19.0, True]"
  ]
}